In [1]:
import os
import openai
import sys
import pandas as pd
import pprint
import requests
import datetime

from dotenv import load_dotenv, find_dotenv

from langchain_openai import OpenAIEmbeddings
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
#from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema.output_parser import StrOutputParser
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

sys.path.append('../..')
pp = pprint.PrettyPrinter(indent=4)
_ = load_dotenv(find_dotenv()) # read local .env file
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["OPENAI_API_KEY"] = "sk-proj-PCfh6KnbKIihSwakajqgT3BlbkFJuutFCLsB3pH5h5Rjmskj"
openai.api_key  = os.environ["OPENAI_API_KEY"] 

## Model declare
llm_model = "gpt-3.5-turbo"
#llm_model = "gpt-4"
llm = ChatOpenAI(model_name=llm_model, temperature=0)
embeddings = OpenAIEmbeddings()


# Vector database And RAG

In [2]:

docs = []
df = pd.read_csv("./data/filtered_sd_housing.csv")
for idx, row in df.iterrows():
    page_content = f"Neighborhood:{row['Neighborhood']}\n\
Price: {row['Price']}\n\
Bedrooms: {row['Bedrooms']}\n\
Bathrooms: {row['Bathrooms']}\n\
House Size:{row['House Size']} \n\
Location:{row['Location']}\n \
Phone Number:{row['Phone Number']}\n\
Construction Year:{row['Construction Year']}\n\
Neighborhood Description: {row['Neighborhood Description']}"
    metadata = {"neighborhood": row["Neighborhood"], "price": row["Price"]}
    doc = Document(page_content=page_content)
    docs.append(doc)

# Split

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)
persist_directory = 'data/chroma/'
splits = text_splitter.split_documents(docs)
# vectordb = Chroma.from_documents(
#     documents=splits,
#     embedding=embeddings,
#     persist_directory=persist_directory
# )
# vectordb.persist()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)
question = "Which houses are located at Height"
ans = vectordb.similarity_search(question,k=3)
# ans = vectordb.max_marginal_relevance_search(question,k=2, fetch_k=3)
#pp.pprint(ans[0].page_content)
ans2 = vectordb.max_marginal_relevance_search(question,k=5, fetch_k=3)
#pp.pprint(ans2[0].page_content)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************mskj. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

# Tagging and Moderation

In [3]:

class Tagging(BaseModel):
    """Tag the piece of text with particular info."""
    sentiment: str = Field(description="sentiment of text, should be `pos`, `neg`, or `neutral`")
    language: str = Field(description="language of text (should be ISO 639-1 code)")

tagging_functions = [convert_pydantic_to_openai_function(Tagging)]
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a content moderation assistant. Your task is to analyze user messages and determine if they contain any inappropriate, offensive, or harmful content. This includes hate speech, explicit or suggestive content, self-harm, illegal activities, or other sensitive topics. \
     Think carefully, and then tag the text as instructed"),
    ("user", "{input}")
])
model_with_tagging = llm.bind(
    functions=tagging_functions,
    function_call={"name": "Tagging"}
)
tagging_chain = prompt | model_with_tagging | JsonOutputFunctionsParser()
tagging_chain.invoke({"input": "non mi piace questo cibo"})


/home/noah-u18-alien/anaconda3/envs/langchain/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


{'sentiment': 'neg', 'language': 'it'}

# Design Collect Preferences prompt

In [4]:
import os
import re
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, ConversationChain, ConversationalRetrievalChain
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.schema.agent import AgentFinish
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
total_answers = 'A comfortable 3 bedrooms house with a spacious kitchen and a cozy living room. A quiet neighborhood, good local schools, and convenient shopping options. A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads. A balance between suburban tranquility and access to urban amenities like restaurants and theaters.'

In [5]:
# Build promptr
buyer_preferences_template = """
Conversation History:
{conversation_history}

You are a professional and polite real estate agent assistant. Your task is to collect buyer preferences for a property by asking\
all the defaults question bellow. 
REMEMBER: Do not make up the user input. Wait for the user response
The DEFAULT questions are:
1. How big do you want your house to be?
2. What are 3 most important things for you in choosing this property?
3. Which amenities would you like?
4. Which transportation options are important to you?
5. How urban do you want your neighborhood to be?  

if the user answers all questions from conversation history,
summarize in detail the user inputs and formating it as the json form bellow.
Always say "The buyer is finding" at the beginning of the summary.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "next_inputs": string \ summarize of the user inputs
```
REMEMBER: only formating the json object only when all buyer information preferences is collected from conversation history.
Use three sentences maximum. Keep the answer as concise as possible.
User Input:
{input}

"""

test_buyer_preferences_template ="""
conversation history :{conversation_history}
**You are a professional real estate agent and you're meeting with a potential buyer. Your goal is to understand their needs and preferences to find them the perfect home.**

**Here are some questions you can ask the buyer:**

* **Can you tell me a little bit about yourself and what brings you to the market for a new home?**
* **How many bedrooms and bathrooms are you looking for?**
* **Do you have a preferred location or area in mind?**
* **What is your budget for a new home?**
* **What type of property are you interested in (e.g., single-family home, condo, townhouse)?**
* **Are there any specific features or amenities that are important to you (e.g., a pool, a garage, a yard)?**
* **Do you have a preferred move-in timeframe?**
* **Is there anything else you would like to tell me about your ideal home?**

**After asking all of the above questions, format the buyer's responses as a JSON object with the following keys:**

* **next_input:** Summarize the answers of the buyer. It must start with "the buyer want to find". Use at max 4 sentences for the summary
**Here is an example of the formatted JSON object:**

```json
{
  "next_input": "The buyer seeks a comfortable 3-bedroom house with a spacious kitchen and cozy living room. Located in a quiet neighborhood with good schools and shopping, it should also offer a backyard, a two-car garage, and easy access to transportation and amenities. This ideal home combines suburban peace with city convenience."
}

user input:{input}

"""
#says something like "i do not have any more questions" or "thank you" or "good bye"
qa_template = """You are a friendly and professional real estate agent.
Use the following pieces of context to find the best home match for a buyer at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
REMEMBER: Check the chat history to provide context-aware answers to the user's questions.
If the user want to stop the conversation by saying for example "i do not have any more questions" or "thank you" or "good bye", you must say thank you for asking me at the end.
Context: {context}

Chat History:{chat_history}

Question: {question}
Helpful Answer:"""
def createBuyerPreferencesAgent(buyer_preferences_template,llm_model = "gpt-3.5-turbo", temperature = 0):
    """
    Creates an agent for collecting buyer preferences.

    Args:
        buyer_preferences_template (str): The prompt template for buyer preference questions.

    Returns:
        ConversationChain: The ConversationChain agent for collecting buyer preferences.
    """
    prompt_template = PromptTemplate(
        input_variables=["conversation_history", "input"],
        template=buyer_preferences_template
    )

    conv_memory = ConversationBufferMemory(memory_key="conversation_history")

    # Create a ConversationChain with memory and the prompt template
    conv_llm = ChatOpenAI(model_name=llm_model, temperature=temperature)
    preferences_collected_agent = ConversationChain(
        llm=conv_llm,
        memory=conv_memory,
        prompt=prompt_template,
        verbose=False
    )
    return preferences_collected_agent

# Build prompt
qa_template = """You are a friendly and professional real estate agent.
Use the following pieces of context to find the best home match for a buyer at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
REMEMBER: Check the chat history to provide context-aware answers to the user's questions.
If the user says something like "i do not have any more questions" or "thank you" or "good bye", you must say thank you for asking me at the end.
Context: {context}

Chat History:{chat_history}

Question: {question}
Helpful Answer:"""

def createQADataAgent(qa_template, vectordb, llm_model = "gpt-3.5-turbo", temperature = 0):
    """
    Creates an agent for answering questions based on a data source.

    Args:
        qa_template (str): The prompt template for asking and combining information.

    Returns:
        ConversationalRetrievalChain: The ConversationalRetrievalChain agent for answering questions.
    """
    QA_CHAIN_PROMPT = PromptTemplate.from_template(qa_template)

    # Set up memory
    memory_data = ConversationBufferMemory(
        memory_key="chat_history",
        input_key="question",
        output_key="answer",
        return_messages=True
    )

    # Set up LLM
    qa_llm = ChatOpenAI(model_name=llm_model, temperature= temperature)

    # Create the ConversationalRetrievalChain
    qa_data_agent = ConversationalRetrievalChain.from_llm(
        llm=qa_llm,
        retriever=vectordb.as_retriever(),
        memory=memory_data,
        combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT},
        verbose = False
    )
    return qa_data_agent



In [10]:

# class BuyerChatStateMachine:
#     """
#     A state machine that manages a conversation with a buyer to collect preferences and find matching properties.

#     This class implements a state machine with three states:
#     1. collecting_preferences: Gather buyer preferences
#     2. finding_match: Search for properties matching the buyer's preferences
#     3. finished: End the conversation

#     Attributes:
#         preferences_collected_agent: An agent for collecting user preferences.
#         qa_data_agent: An agent for querying property data.
#         qa_data_conversation: An agent for handling follow-up conversations about properties.
#         state (str): The current state of the conversation.
#         next_input (str): The next input to be processed.
#         max_number_conversation (int): Maximum number of allowed conversation turns.
#         conversation_count (int): Current number of conversation turns.
#     """

#     def __init__(self, preferences_collected_agent, qa_data_agent, max_number_conversation=10):
#         """
#         Initialize the BuyerChatStateMachine.

#         Args:
#             preferences_collected_agent: Agent for collecting user preferences.
#             qa_data_agent: Agent for querying property data and follow up question.
#             qa_data_conversation: Agent for handling follow-up property conversations.
#             max_number_conversation (int, optional): Maximum number of conversation turns. Defaults to 10.
#         """
#         self.preferences_collected_agent = preferences_collected_agent
#         self.qa_data_agent = qa_data_agent
#         self.state = "collecting_preferences"
#         self.next_input = ""
#         self.max_number_conversation = max_number_conversation
#         self.conversation_count = 0

#     def run(self):
#         """
#         Run the state machine to manage the buyer conversation.

#         This method controls the flow of the conversation, transitioning between states
#         and ensuring the conversation doesn't exceed the maximum number of turns.
#         """
#         print("Please ask me anything: ")
#         while self.conversation_count < self.max_number_conversation:
#             self.conversation_count += 1
#             if self.state == "collecting_preferences":
#                 self._handleCollectingPreferences()
#             elif self.state == "finding_match":
#                 self._handleFindingMatch()
#             elif self.state == "finished":
#                 break
#             else:
#                 print("Invalid state encountered")
#                 break

#         if self.conversation_count >= self.max_number_conversation:
#             print(f"Reached maximum number of conversations ({self.max_number_conversation}). Ending chat.")
#         else:
#             print("Chat finished. Thank you!")

#     def _handleCollectingPreferences(self):
#         """
#         Handle the 'collecting_preferences' state.

#         Collects user preferences and checks if all required preferences are gathered.
#         If so, transitions to the 'finding_match' state.
#         """
#         buyer_preferences = self._collectUserPreferences()
#         is_all_collected = self._isAllPreferencesCollected(buyer_preferences)
#         if is_all_collected[0]:
#             self.state = "finding_match"
#             self.next_input = is_all_collected[1]
#             return
#         print(buyer_preferences)
#     def _handleFindingMatch(self):
#         """
#         Handle the 'finding_match' state.

#         Queries the property database based on collected preferences and presents results to the user.
#         If the user is satisfied, transitions to the 'finished' state. Otherwise, prompts for more input.
#         """
#         response_check = self.qa_data_agent({"question": self.next_input})
#         print(response_check['answer'])
#         if "thank you for asking me" in response_check['answer'].lower():
#             self.state = "finished"
#             return
#         self.next_input = input(f"User input (Conversation {self.conversation_count}/{self.max_number_conversation}): ")

#     def _collectUserPreferences(self):
#         """
#         Collect user preferences using the preferences_collected_agent.

#         Returns:
#             str: The collected user preferences.
#         """
#         user_input = input(f"User input (Conversation {self.conversation_count}/{self.max_number_conversation}): ")
#         return self.preferences_collected_agent.predict(input=user_input)

#     def _isAllPreferencesCollected(self, user_input):
#         """
#         Check if all required user preferences have been collected.

#         Args:
#             user_input (str): The user's input to be checked.

#         Returns:
#             list: A list containing a boolean indicating if all preferences are collected,
#                   and the next input if available.
#         """
#         pattern = r'"next_inputs":\s*"([^"]*)"'
#         match = re.search(pattern, user_input)
#         if match:
#             return [True, match.group(1)]
#         return [False, "Not Finished"]

# # Usage example:stop
# # preferences_collected_agent = ...  # Initialize your agent
# # qa_data_agent = ...  # Initialize your agent
# # qa_data_conversation = ...  # Initialize your agent
# # 
# # state_machine = BuyerChatStateMachine(preferences_collected_agent, qa_data_agent, qa_data_conversation, max_number_conversation=15)
# # state_machine.run()

# total_answers

'A comfortable 3 bedrooms house with a spacious kitchen and a cozy living room. A quiet neighborhood, good local schools, and convenient shopping options. A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads. A balance between suburban tranquility and access to urban amenities like restaurants and theaters.'

In [38]:

class BuyerChatStateMachine:
    """
    A state machine that manages a conversation with a buyer to collect preferences and find matching properties.

    This class implements a state machine with three states:
    1. collecting_preferences: Gather buyer preferences
    2. finding_match: Search for properties matching the buyer's preferences
    3. finished: End the conversation

    Attributes:
        preferences_collected_agent: An agent for collecting user preferences.
        qa_data_agent: An agent for querying property data.
        qa_data_conversation: An agent for handling follow-up conversations about properties.
        state (str): The current state of the conversation.
        next_input (str): The next input to be processed.
        max_number_conversation (int): Maximum number of allowed conversation turns.
        conversation_count (int): Current number of conversation turns.
    """

    def __init__(self, preferences_collected_agent, qa_data_agent, max_number_conversation=10):
        """
        Initialize the BuyerChatStateMachine.

        Args:
            preferences_collected_agent: Agent for collecting user preferences.
            qa_data_agent: Agent for querying property data and follow up question.
            qa_data_conversation: Agent for handling follow-up property conversations.
            max_number_conversation (int, optional): Maximum number of conversation turns. Defaults to 10.
        """
        self.preferences_collected_agent = preferences_collected_agent
        self.qa_data_agent = qa_data_agent
        self.state = "collecting_preferences"
        self.response = ""
        self.max_number_conversation = max_number_conversation
        self.conversation_count = 0
        self.rag_first_trigger = True

    def generateAnswer(self, user_input):
        """
        Run the state machine to manage the buyer conversation.

        This method controls the flow of the conversation, transitioning between states
        and ensuring the conversation doesn't exceed the maximum number of turns.
        """
        print("Please ask me anything: ")
        if (self.conversation_count < self.max_number_conversation):
            self.conversation_count += 1
            if self.state == "collecting_preferences":
                self._handleCollectingPreferences(user_input)
            elif self.state == "finding_match":
                self._handleFindingMatch(user_input)
            elif self.state == "finished":
                self.response = "Finish State Reach"
            else:
                self.response = "Invalid state encountered"
        else:
            self.response = f"Reached maximum number of conversations ({self.max_number_conversation}). Ending chat."
        # else:
        #     print("Chat finished. Thank you!")
        return self.response

    def _handleCollectingPreferences(self, user_input):
        """
        Handle the 'collecting_preferences' state.

        Collects user preferences and checks if all required preferences are gathered.
        If so, transitions to the 'finding_match' state.
        """
        self.response = self.preferences_collected_agent.predict(input=user_input)
        is_all_collected = self._isAllPreferencesCollected( self.response )
        if is_all_collected[0]:
            self.state = "finding_match"
            if(self.rag_first_trigger):
                self.response = self.qa_data_agent({"question": is_all_collected[1]})['answer']
                self.rag_first_trigger = False

    def _handleFindingMatch(self,user_input):
        """
        Handle the 'finding_match' state.

        Queries the property database based on collected preferences and presents results to the user.
        If the user is satisfied, transitions to the 'finished' state. Otherwise, prompts for more input.
        """
        self.response = self.qa_data_agent({"question": user_input})['answer']
        if "thank you for asking me" in self.response.lower():
            self.rag_first_trigger = True
            self.state = "finished"
        # self.next_input = input(f"User input (Conversation {self.conversation_count}/{self.max_number_conversation}): ")


    def _isAllPreferencesCollected(self, user_input):
        """
        Check if all required user preferences have been collected.

        Args:
            user_input (str): The user's input to be checked.

        Returns:
            list: A list containing a boolean indicating if all preferences are collected,
                  and the next input if available.
        """
        pattern = r'"next_inputs":\s*"([^"]*)"'
        match = re.search(pattern, user_input)
        if match:
            return [True, match.group(1)]
        return [False, "Not Finished"]



total_answers

'A comfortable 3 bedrooms house with a spacious kitchen and a cozy living room. A quiet neighborhood, good local schools, and convenient shopping options. A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads. A balance between suburban tranquility and access to urban amenities like restaurants and theaters.'

In [41]:

# Usage example:
preferences_collected_agent = createBuyerPreferencesAgent(buyer_preferences_template=buyer_preferences_template)  # Initialize your agent
qa_data_agent = createQADataAgent(qa_template=qa_template, vectordb=vectordb)  # Initialize your agent
home_match_bot = BuyerChatStateMachine(preferences_collected_agent=preferences_collected_agent,qa_data_agent= qa_data_agent, max_number_conversation=10)
# home_match_bot.generateAnswer("Hi")

In [45]:
import gradio as gr
# Function to generate answer

# Function to generate answer
def generateBotAnswer(message, history):
    history = history or []
    response = home_match_bot.generateAnswer(message)  # Note: Fixed typo in method name
    history.append((message, response))
    return history, history

# # Create Gradio interface
# with gr.Blocks() as block:
#     gr.Markdown("""<h1><center>Home Match Bot</center></h1>""")
#     chatbot = gr.Chatbot()
#     message = gr.Textbox(placeholder="Ask me a question about houses")
#     state = gr.State([])  # Initialize with an empty list
#     submit = gr.Button("Send")
    
#     # Set up the click event
#     submit.click(
#         generate_bot_answer, 
#         inputs=[message, state], 
#         outputs=[chatbot, state]
#     )

# # Launch the interface
# block.launch(debug=True)

# Set up chat bot interface
# Check if ChatInterface is available
if hasattr(gr, 'ChatInterface'):
    # Use ChatInterface for newer Gradio versions
    answer_bot = gr.ChatInterface(
        generateBotAnswer,
        chatbot=gr.Chatbot(height=300),
        textbox=gr.Textbox(placeholder="Ask me a question about finding home", container=False, scale=7),
        title="Perfect Home Match ChatBot",
        description="Ask The Real Estate Agent!",
        theme="soft",
        examples=["location", "neighborhood", "price range", "house size", "contact"],
        cache_examples=False,
        retry_btn=None,
        undo_btn=None,
        clear_btn=None,
        submit_btn="Ask"
    )
else:
    # Fallback for older Gradio versions
    with gr.Blocks(theme="soft") as answer_bot:
        gr.Markdown("# Perfect Home Match ChatBot")
        gr.Markdown("Ask The Real Estate Agent!")
        
        chatbot = gr.Chatbot(height=300)
        msg = gr.Textbox(placeholder="Ask me a question about finding home", container=False, scale=7)
        
        with gr.Row():
            submit = gr.Button("Ask")
        
        examples = gr.Examples(
            examples=["location", "neighborhood", "price range", "house size", "contact"],
            inputs=msg
        )
        
        def respond(message, chat_history):
            bot_message = generateBotAnswer(message, chat_history)
            chat_history.append((message, bot_message))
            return "", chat_history
        
        submit.click(respond, [msg, chatbot], [msg, chatbot])
        msg.submit(respond, [msg, chatbot], [msg, chatbot])

# Launch the interface
answer_bot.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://18db9002d15c8af954.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/home/noah-u18-alien/anaconda3/envs/langchain/lib/python3.9/site-packages/uvicorn/protocols/http/httptools_impl.py", line 399, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/home/noah-u18-alien/anaconda3/envs/langchain/lib/python3.9/site-packages/uvicorn/middleware/proxy_headers.py", line 70, in __call__
    return await self.app(scope, receive, send)
  File "/home/noah-u18-alien/anaconda3/envs/langchain/lib/python3.9/site-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/home/noah-u18-alien/anaconda3/envs/langchain/lib/python3.9/site-packages/starlette/applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/home/noah-u18-alien/anaconda3/envs/langchain/lib/python3.9/site-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/hom

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/home/noah-u18-alien/anaconda3/envs/langchain/lib/python3.9/site-packages/uvicorn/protocols/http/httptools_impl.py", line 399, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/home/noah-u18-alien/anaconda3/envs/langchain/lib/python3.9/site-packages/uvicorn/middleware/proxy_headers.py", line 70, in __call__
    return await self.app(scope, receive, send)
  File "/home/noah-u18-alien/anaconda3/envs/langchain/lib/python3.9/site-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/home/noah-u18-alien/anaconda3/envs/langchain/lib/python3.9/site-packages/starlette/applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/home/noah-u18-alien/anaconda3/envs/langchain/lib/python3.9/site-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/hom

'A comfortable 3 bedrooms house with a spacious kitchen and a cozy living room. A quiet neighborhood, good local schools, and convenient shopping options. A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads. A balance between suburban tranquility and access to urban amenities like restaurants and theaters.'

# Tools and Routing Function

In [109]:
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.llm import LLMChain

# Build prompts
question_prompt_template = """Use the following pieces of context to find the best home match for a buyer. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible.
{context_str}
Question: {question}
Helpful Answer:"""
QUESTION_PROMPT = PromptTemplate(
    template=question_prompt_template, input_variables=["context_str", "question"]
)

refine_template = """The original question is as follows: {question}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer
(only if needed) with some more context below.
------------
{context_str}
------------
Given the new context, refine the original answer to better answer the question. 
If the context isn't useful, return the original answer. Always say "thanks for asking!" at the end of the answer."""
REFINE_PROMPT = PromptTemplate(
    template=refine_template,
    input_variables=["question", "existing_answer", "context_str"],
)

# Create a prompt for question generation
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:""")

memory_data = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="question",
    output_key="answer",
    return_messages=True
)

# Create the refine chain
refine_chain = load_qa_chain(
    llm,
    chain_type="refine",
    question_prompt=QUESTION_PROMPT,
    refine_prompt=REFINE_PROMPT,
)

# Create a question generator chain
question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT)

# Create the ConversationalRetrievalChain
qa_chain = ConversationalRetrievalChain(
    retriever=vectordb.as_retriever(),
    memory=memory_data,
    combine_docs_chain=refine_chain,
    question_generator=question_generator,
    verbose = True
)

# To use the chain:
query = "What's the best home match for a family of four?"
result = qa_chain({"question": query})
print(result['answer'])



> Entering new ConversationalRetrievalChain chain...

> Finished chain.
The best home match for a family of four would be this charming single-story 4 bedroom, 2 bathroom home located at 5045 Park West Ave in the highly sought-after neighborhood of Clairemont Mesa, San Diego, CA 92117. This home is a blank canvas waiting for you to make it your own, with a newer roof, a spacious lot, and the opportunity to customize to your liking. The neighborhood offers easy access to amenities like bars, restaurants, coffee shops, and parks, making it a convenient and enjoyable place to live. Priced at $1,093,000, this home is a must-see for any family looking for a cozy and welcoming space in a great location. Professional pictures and video are coming soon. Thanks for asking!


In [110]:
# To use the chain:
query = "What's the address of this house?"
result = qa_chain({"question": query})
print(result['answer'])



> Entering new ConversationalRetrievalChain chain...

> Finished chain.
5381 Vergara St, San Diego, CA 92117

Thanks for asking!


In [3]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{user_input}}

<< OUTPUT (remember to include the ```json)>>"""


router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["user_input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt, verbose = True)
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

buyer_preferences_template = """You are a very professional real estate agent assistant. \
Your task is to collect best matches for a property from user preferences.
try to make it clear to call the database\
"""
prompt_infos = [
    {
        "name": "buyer_preferences_finding", 
        "description": "Finding match home from byer preferences", 
        "prompt_template": buyer_preferences_template
    }
]


destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt, memory= ConversationBufferMemory())
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
default_prompt = ChatPromptTemplate.from_template("{user_input}")
default_memory = ConversationBufferMemory()
default_chain = LLMChain(llm=llm, prompt=default_prompt, memory=default_memory)


NameError: name 'destinations_str' is not defined